In [1]:
import pandas as pd

In [2]:
bank = pd.read_csv('bank.csv', sep=';')
bank_add_full = pd.read_csv('bank-additional-full.csv', sep=';')
bank_add = pd.read_csv('bank-additional.csv', sep=';')
bank_full = pd.read_csv('bank-full.csv', sep=';')

# 1. Logistic Regression

In [3]:
# Step 1: Import Packages, Functions, and Classes
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
bank_add_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [5]:
bank_add_full['education'].unique()

array(['basic.4y', 'high.school', 'basic.6y', 'basic.9y',
       'professional.course', 'unknown', 'university.degree',
       'illiterate'], dtype=object)

In [6]:
bank_add_full['education']=np.where(bank_add_full['education'] =='basic.9y', 'Basic', bank_add_full['education'])
bank_add_full['education']=np.where(bank_add_full['education'] =='basic.6y', 'Basic', bank_add_full['education'])
bank_add_full['education']=np.where(bank_add_full['education'] =='basic.4y', 'Basic', bank_add_full['education'])

In [7]:
# data exploration
bank_add_full['y'].value_counts()

no     36548
yes     4640
Name: y, dtype: int64

In [8]:
count_no_sub = len(bank_add_full[bank_add_full['y']=='no'])
count_sub = len(bank_add_full[bank_add_full['y']=='yes'])
pct_of_no_sub = count_no_sub/(count_no_sub+count_sub)
print("percentage of no subscription is", pct_of_no_sub*100)
pct_of_sub = count_sub/(count_no_sub+count_sub)
print("percentage of subscription", pct_of_sub*100)

percentage of no subscription is 88.73458288821988
percentage of subscription 11.265417111780131


dapat terlihat bahwa rasio dari no-subscription dan subscription tidak imbang sebesar 89:11

In [9]:
# creaet variabel dummy
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(bank_add_full[var], prefix=var)
    data=bank_add_full.join(cat_list)
    bank_add_full=data
cat_vars=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
data_vars=bank_add_full.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

In [10]:
data_final=bank_add_full[to_keep]
data_final.columns.values

array(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired',
       'job_self-employed', 'job_services', 'job_student',
       'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_Basic', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown', 'default_no',
       'default_unknown', 'default_yes', 'housing_no', 'housing_unknown',
       'housing_yes', 'loan_no', 'loan_unknown', 'loan_yes',
       'contact_cellular', 'contact_telephone', 'month_apr', 'month_aug',
       'month_dec', 'month_jul', 'month_jun', 'month_mar', 'month_may',
       'month_nov', 'month_oct', 'month_sep', 'day_of_week_fri',

In [11]:
X = data_final.loc[:, data_final.columns != 'y']
y = data_final.loc[:, data_final.columns == 'y']

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [12]:
# create a model
model = LogisticRegression(solver='liblinear', C=0.05, multi_class='ovr', random_state=0)
model.fit(X_train, y_train)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression(C=0.05, multi_class='ovr', random_state=0,
                   solver='liblinear')

In [13]:
# step 4 : evaluate the model
X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)

model.score(X_train, y_train)

0.9106864139294509

Dari hasil observasi dapat diketahui bahwa akurasi model dari data yang diklasifikasikan benar sebesar 91%

### Confusion Matrix

In [14]:
# confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[10672   297]
 [  793   595]]


In [15]:
# mendapatkan laporan yg lebih komprehensif dengan classification_report()
print(classification_report(y, model.predict(X)))

              precision    recall  f1-score   support

          no       0.89      1.00      0.94     36548
         yes       0.63      0.03      0.05      4640

    accuracy                           0.89     41188
   macro avg       0.76      0.51      0.50     41188
weighted avg       0.86      0.89      0.84     41188



In [16]:
# improve the model
model = LogisticRegression(solver='liblinear', C=10.0, multi_class='ovr', random_state=0)
model.fit(X,y)

print("intercept :",model.intercept_)
print("coefisien :",model.coef_)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


intercept : [0.00302732]
coefisien : [[ 5.26431096e-03  4.60330238e-03 -1.06878397e-01 -1.68398567e-03
  -1.62908153e-01 -2.39956980e-01  3.48767305e-01  2.82496746e-02
  -1.82839510e-01 -6.47990904e-03  6.91869786e-02 -1.64667119e-01
  -1.96793565e-02 -2.63967935e-03 -4.52686223e-04  8.14824817e-02
  -6.51179312e-03 -5.40419069e-02  5.84342189e-02  2.98020223e-02
   1.14530676e-02  6.61087196e-04 -1.34479000e-02 -7.58950867e-02
   9.22219167e-02  1.48385056e-04 -1.25484995e-01 -4.84499561e-02
   2.17571126e-03  1.98697267e-02  1.35369418e-01  1.95474106e-02
   1.39007107e-01 -1.35917904e-01 -6.18879476e-05  2.04863164e-03
  -5.13734148e-03  6.11602485e-03  2.85172334e-02 -5.13734148e-03
  -2.03525769e-02  2.00357369e-01 -1.97330054e-01  2.74615009e-02
   1.03376509e-01  7.02729329e-03  1.42876461e-01  1.15967420e-01
   1.82434149e-01 -5.33988538e-01 -4.76730919e-02  2.45442764e-02
  -1.89986645e-02 -3.09025172e-02 -7.67009115e-02  7.11151182e-03
   4.82509532e-02  5.52682787e-02 -1.72

In [17]:
print(model.predict_proba(X))

[[0.98241313 0.01758687]
 [0.99169417 0.00830583]
 [0.98599583 0.01400417]
 ...
 [0.64976343 0.35023657]
 [0.3967333  0.6032667 ]
 [0.73028111 0.26971889]]


In [18]:
print("y pred :",model.predict(X))

y pred : ['no' 'no' 'no' ... 'no' 'yes' 'no']


In [19]:
print("nilai akurasi:",model.score(X,y))

nilai akurasi: 0.9106050305914344


**Diperoleh nilai akurasi model hasil pengamatan sebesar 91%**

# 2. K-Nearest Neighbors

In [20]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


In [21]:
# change variable into categorical
knn = bank.select_dtypes(include=['object']).copy()
nRows, nCols = knn.shape
for myIndex in range(0,nCols):
    headerName = knn.columns[myIndex]
    knn[headerName] = knn[headerName].astype("category")
    knn[headerName] = knn[headerName].cat.codes
    bank[headerName] = knn[headerName]
knn_numeric = bank
knn_numeric.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,10,1,0,0,1787,0,0,0,19,10,79,1,-1,0,3,0
1,33,7,1,1,0,4789,1,1,0,11,8,220,1,339,4,0,0
2,35,4,2,2,0,1350,1,0,0,16,0,185,1,330,1,0,0
3,30,4,1,2,0,1476,1,1,2,3,6,199,4,-1,0,3,0
4,59,1,1,1,0,0,1,0,2,5,8,226,1,-1,0,3,0


In [22]:
x = knn_numeric.loc[:, knn_numeric.columns != 'y']
y = knn_numeric.loc[:, knn_numeric.columns == 'y']

In [23]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)

# spliting data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 123)

# scaling data
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

model.fit(x_train,y_train)
y_pred = model.predict(x_test)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [24]:
model.predict_proba(x_test)

array([[1. , 0. ],
       [0.6, 0.4],
       [1. , 0. ],
       ...,
       [1. , 0. ],
       [0.8, 0.2],
       [1. , 0. ]])

### Confusion Matrix

In [25]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))

[[788  20]
 [ 76  21]]


**Artinya :**
- Terdapat 788 true negatif pada 788 observasi pertama
- Terdapat 76 false negatif
- Terdapat 20 false positif pada pengamatan
- Terdapat 21 true positif pada 21 observasi terakhir

In [26]:
# evaluate the model
from sklearn import metrics

# model akurasi
print("Accuracy:",metrics.accuracy_score(y_test,y_pred))

Accuracy: 0.8939226519337017


**Diperoleh nilai akurasi model hasil pengamatan sebesar 85.55%**

# 3. Support Vector Machine

In [27]:
feature_cols = ['age','job','marital','education','default','balance','housing','loan',
                 'contact','day','month','duration','campaign','pdays','previous','poutcome']

X = bank[feature_cols]
y = bank.loc[:, bank.columns == 'y']

# spliting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 109)

In [ ]:
# generating model
from sklearn import svm

# ctreate as svm Classifier
clf = svm.SVC(kernel='linear')

# train the model
clf.fit(X_train, y_train)

# predict the model
y_pred = clf.predict(X_test)

### Confusion Matrix

In [28]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [1357, 905]

**Artinya :**
- Terdapat 1093 true negatif
- Terdapat 77 false negatif
- Terdapat 119 false positif pada pengamatan
- Terdapat 68 true positif pada 21 observasi terakhir

In [ ]:
# evaluate the model
from sklearn import metrics

# model akurasi
print("Accuracy:",metrics.accuracy_score(y_test,y_pred))

**Diperoleh nilai akurasi model hasil pengamatan sebesar 87.9%**

# 4. Decision Tree

In [ ]:
feature_cols = ['age','job','marital','education','default','balance','housing','loan',
                 'contact','day','month','duration','campaign','pdays','previous','poutcome']
X = bank[feature_cols]
y = bank.loc[:, bank.columns == 'y']

# spliting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

In [ ]:
# training and making prediction
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
y_pred

### Confusion Matrix

In [ ]:
# evaluate model
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))

**Artinya :**
- Terdapat 1093 true negatif
- Terdapat 77 false negatif
- Terdapat 119 false positif pada pengamatan
- Terdapat 68 true positif pada 21 observasi terakhir

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn import metrics

# model accuracy
print("Accuracy:",metrics.accuracy_score(y_test,y_pred))

**Dapat diketahui bahwa dari total 1357 data observasi terdapat 158 missclasified dengan akurasi model sebesar 88%**

In [ ]:
# visualisasi Decision Tree
features = bank.loc[:, bank.columns != 'y']

import sklearn.tree as tree
import pydotplus
from six import StringIO
from IPython.display import Image

dot_data = StringIO()
tree.export_graphviz(classifier,
                     out_file=dot_data,
                     class_names=['0', '1'], 
                     feature_names=feature_cols, 
                     filled=True,
                     rounded=True, 
                     special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

# 5. Random Forest

In [ ]:
feature_cols = ['age','job','marital','education','default','balance','housing','loan',
                 'contact','day','month','duration','campaign','pdays','previous','poutcome']
X = bank[feature_cols]
y = bank.loc[:, bank.columns == 'y']

# spliting data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# create a gaussian classifier
rf=RandomForestClassifier(n_estimators=100)

# train the model
rf.fit(X_train,y_train)

# predict
y_pred = rf.predict(X_test)

### Confusion Matrix

In [ ]:
# confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))

**Artinya :**
- Terdapat 1093 true negatif
- Terdapat 77 false negatif
- Terdapat 119 false positif pada pengamatan
- Terdapat 68 true positif pada 21 observasi terakhir

In [ ]:
from sklearn import metrics

# model accuracy
print("Accuracy:",metrics.accuracy_score(y_test,y_pred))

**Diperoleh nilai akurasi model hasil pengamatan sebesar 90.7%**

# 6. Naive Bayes

In [ ]:
x = bank.loc[:, bank.columns != 'y']
y = bank.loc[:, bank.columns == 'y']

# spliting data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 123)

In [ ]:
# Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

# create gaussian classifier
model = GaussianNB()

# train the model
model.fit(x_train,y_train)

# predict output
y_pred = model.predict(x_test)
print(y_pred)

### Confusion Matrix

In [ ]:
# confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))

**Artinya :**
- Terdapat 1093 true negatif
- Terdapat 77 false negatif
- Terdapat 119 false positif pada pengamatan
- Terdapat 68 true positif pada 21 observasi terakhir

In [ ]:
from sklearn import metrics

# model accuracy
print("Accuracy:",metrics.accuracy_score(y_test,y_pred))

Dapat diketahui bahwa akurasi hasil observasi data sebesar 85.55%

# 7. Algorithm Analysis

Setelah dilakukan analasis dengan berbagai metode pengklasifikan, diperoleh nilai akurasi dari masing-masing model sebagai berikut:

1. Logistic Regregression = 91%
2. K-Nearest Neighbors = 85.%
3. Support Vector Machine = 87.9%
4. Decision Tree = 88%
5. Random Forest = 90.7%
6. Naive Bayes = 85.55%

Berdasarkan besaran nilai akurasi pada tiap-tiap model, didapatkan model terbaik yaitu model Logistic Regression dengan nilai akurasi sebesar 91%